# Image Classification with Tensors


This notebook demonstrates image classification using tensors and Tucker decomposition.

We start with a set of $K$ labeled training images. Each image can be represented as a tensor:

\begin{align*}
X^{(k)} \in \mathbb{R}^{I_{1} \times I_{2} \times \cdots \times I_{N}}, 
\quad (k = 1, 2, \ldots, K)
\end{align*}
These training images correspond to $C$ categories/classes. 

Similarly, we represent our $T$ test images as:
\begin{align*}
\dot{X}^{(t)} \in \mathbb{R}^{I_{1} \times I_{2} \times \cdots \times I_{N}}, 
\quad (t = 1, 2, \ldots, T).
\end{align*}

The goal is to assign labels to these test images.

The classification algorithm:

1. **Learn basis matrices from training data:**
     For each sample $X^{(k)}$, we decompose it into a smaller core tensor $\mathcal{G}^{(k)}$ and factor matrices:
        
    \begin{align*}
            X^{(k)} \approx \mathcal{G}^{(k)} 
            \times_{1} A^{(1)} 
            \times_{2} A^{(2)} 
            \times_{3} \cdots 
            \times_{N} A^{(N)}, 
            \quad (k = 1, 2, \ldots, K)
        \end{align*}

    where the core tensor $\mathcal{G}^{(k)} \in \mathbb{R}^{J_{1} \times J_{2} \times \cdots \times J_{N}}$ captures the features of $X^{(k)}$ in a much lower-dimensional subspace.

2. **Extract features for test samples:**
     Project the test tensors onto the learned basis factors:

    \begin{align*}
        \dot{X}^{(t)} 
        \times_{1} (A^{(1)})^{T} 
        \times_{2} (A^{(2)})^{T} 
        \times_{3} \cdots 
        \times_{N} (A^{(N)})^{T}.
    \end{align*}

3. **Classify:** Compare the extracted test features with the training features to assign labels.


# Dataset used

We test this approach on a small dataset of 28 training images:

- 14 Cats  
- 14 Birds  

Thus, the number of classes is $C = 2$.  

First, we apply Tucker decomposition on grayscale images (third-order tensor) with ranks:  

$$
R_{1} = 10, \quad R_{2} = 10, \quad R_{3} = 28
$$

Then, we extend the method to color images (fourth-order tensor) with ranks:  

$$
R_{1} = 10, \quad R_{2} = 10, \quad R_{3} = 3, \quad R_{4} = 28
$$


In [ ]:
#loading libraries:
import numpy as np
from scipy.io import loadmat
import tensorly as tl
import os
import cv2
from tensorly.decomposition import tucker
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

tl.set_backend('numpy')

**Prediction on greyscale images:**

In [ ]:
# Function to load images from a folder, convert to grayscale, and resize
def load_images(folder, prefix, n, size=(100, 100)):
    images = []
    for i in range(1, n+1):
        img = cv2.imread(os.path.join(folder, f'{prefix}{i}.jpg'))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, size)
        images.append(resized)
    return np.array(images)

# Loading training images and converting to tensor:
train_images = load_images('CatsBirds', 'train', 28) 
train_tensor = np.moveaxis(train_images, 0, -1)
# print(train_tensor.shape)
# print(train_tensor[-1])

# Performing Tucker decomposition on the training tensor:
R = (10, 10, 28) 
core, factors = tucker(train_tensor, rank=R)

# print(core.shape)
# print(type(factors))
# print(len(factors))
# print(factors[0])

# Stacking the training tensor:
train_features = core.reshape(-1, 28).T 
#train_features.shape

# Loading test images and converting to tensor:
# Note: Adjust the number of test images as needed
test_images = load_images('CatsBirds', 'Test', 12)
test_features = []
for img in test_images:
  
    projected = factors[0].T @ img @ factors[1]
    test_features.append(projected.flatten())

# Converting test features to numpy array:
test_features = np.array(test_features)

# Loading training labels:
train_labels = loadmat('train_lab-1.mat')['train'].flatten()
#print(train_labels)

# Training a Random Forest classifier:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_features, train_labels)

# Predicting on test features:
pred_labels = rf.predict(test_features)
print(F'predicted_labels:\n {pred_labels}')

# Loading true labels for the test set:
true_labels = loadmat('test_lab-1.mat')['test'].flatten()
print(f'true_labels:\n {true_labels}')

# Evaluating the model:
conf = confusion_matrix(true_labels, predicted_labels)
print(f'confusion matrix:\n {conf}')

accuracy = accuracy_score(true_labels, predicted_labels)
print(f'accuracy: {accuracy}')

predicted_labels:
 [0 1 0 1 0 0 1 0 1 0 0 1]
true_labels:
 [1 1 0 1 0 1 0 0 1 0 0 1]
confusion matrix:
 [[5 1]
 [2 4]]
accuracy: 0.75


**Prediction on RGB images:**

In [56]:
# Function to load images from a folder, convert to RGB, and resize
def load_images_rgb(folder, prefix, n, size=(100, 100)):
    images = []
    for i in range(1, n+1):
        img = cv2.imread(os.path.join(folder, f'{prefix}{i}.jpg'))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        images.append(img)
    return np.array(images)

# Loading training images and converting to RGB tensor:
train_images = load_images_rgb('CatsBirds', 'train', 28)
train_tensor = np.moveaxis(train_images, 0, -1) 

# Performing Tucker decomposition on the RGB training tensor:
R = (10, 10, 3, 28)
core, factors = tucker(train_tensor, rank=R)

# Stacking the training tensor:
train_features = core.reshape(-1, 28).T  
#print(train_features.shape)

# Loading test images and converting to RGB tensor:
test_images = load_images_rgb('CatsBirds', 'Test', 12)

# Projecting the test images using the factors from the Tucker decomposition:
# Note: Adjust the number of test images as needed
test_features = []
for img in test_images:

    proj = np.einsum('ia,jb,kc,ijk->abc', factors[0], factors[1], factors[2], img)
    test_features.append(proj.flatten())

test_features = np.array(test_features) 
#print(test_features.shape)

# Loading training labels:
train_labels = loadmat('train_lab-1.mat')['train'].flatten()

# Training a Random Forest classifier:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(train_features, train_labels)

# Predicting on test features:
pred_labels = rf.predict(test_features)
print(f'predicted_labels:\n {pred_labels}')

# Loading true labels for the test set:
true_labels = loadmat('test_lab-1.mat')['test'].flatten()
print(f'true_labels:\n {true_labels}')

# Evaluating the model:
conf = confusion_matrix(true_labels, pred_labels)
print(f'confusion matrix:\n {conf}')

accuracy = accuracy_score(true_labels, pred_labels)
print(f'accuracy: {accuracy}')

classification_error = 1 - accuracy
print(f'classification error: {classification_error}')


predicted_labels:
 [1 0 1 0 0 1 1 0 0 0 1 1]
true_labels:
 [1 1 0 1 0 1 0 0 1 0 0 1]
confusion matrix:
 [[3 3]
 [3 3]]
accuracy: 0.5
classification error: 0.5
